In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
projects <- c("cesc", "ucs", "ucec")

In [3]:
dset_idx <- 1
save_res <- TRUE

In [46]:
mat_df <- rutils::load_matrisome_df(paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")) %>%
    select(gene_symbol, gene_name, division, category, hgnc_ids)
figo_demg_df <- read_lines(paste0(dirs$analysis_dir, "/gene_lists_extra/", projects[dset_idx], "_figo_demg.txt")) %>%
    as_tibble() %>%
    rename(geneID = value)
deg_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv")) %>%
    select(geneID, baseMean, log2FoldChange, qval) %>%
    rename(de_base_mean = baseMean, de_l2fc = log2FoldChange, de_qval = qval)
waov_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_welch_anova_results.tsv")) %>%
    select(geneID, contains("qval")) %>%
    rename_with(~ str_replace(., "qval", "anova_qval"))
pbc_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_pbc_results.tsv")) %>%
    select(geneID, contains("qval"), contains("cor")) %>%
    rename_with(~ str_replace(., "qval", "cor_qval")) %>%
    rowwise() %>%
    mutate(min_cor_qval = min(figo_stage_1_cor_qval, figo_stage_2_cor_qval, figo_stage_3_cor_qval, figo_stage_4_cor_qval)) %>%
    ungroup()
lasso_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_en_multinom_results.tsv")) %>%
    # All coefs should be the same
    mutate(lasso_coef = figo_stage_1_coefs) %>%
    select(geneID, lasso_coef)
wgcna_mm_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_gene_mm_gs.tsv")) %>%
    select(geneID, module, mm_pval)
wgcna_eg_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_eigengene_traits.tsv")) %>%
    rowwise() %>%
    mutate(eg_figo_qval_min = min(figo_stage_1_qval, figo_stage_2_qval, figo_stage_3_qval, figo_stage_4_qval)) %>%
    ungroup() %>%
    select(-starts_with("vital"), -contains("pval"), -contains("cor")) %>%
    rename_with(~ str_replace(., "qval", "wgcna_qval"))


-- Column specification ------------------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double(),
  qval = col_double()
)



-- Column specification ------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  pval = col_double(),
  padj = c

In [47]:
meta_df <- figo_demg_df %>%
    inner_join(mat_df, by = c("geneID" = "gene_symbol")) %>%
    inner_join(deg_df, by = "geneID") %>%
    left_join(waov_df, by = "geneID") %>%
    left_join(pbc_df, by = "geneID") %>%
    left_join(lasso_df, by = "geneID") %>%
    inner_join(wgcna_mm_df, by = "geneID") %>%
    inner_join(wgcna_eg_df, by = "module") %>%
    replace_na(list(lasso_coef = 0))

In [57]:
meta_df %>%
    filter(module == "blue") %>%
    filter(anova_qval < 0.05 | min_cor_qval < 0.05 | lasso_coef != 0) %>%
    select(geneID, anova_qval, min_cor_qval, figo_stage_1_cor_qval, figo_stage_2_cor_qval, figo_stage_3_cor_qval, figo_stage_4_cor_qval, lasso_coef, module, eg_figo_wgcna_qval_min)

geneID,anova_qval,min_cor_qval,figo_stage_1_cor_qval,figo_stage_2_cor_qval,figo_stage_3_cor_qval,figo_stage_4_cor_qval,lasso_coef,module,eg_figo_wgcna_qval_min
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
HRNR,0.30882108,0.13429176,0.13429176,0.9948641,0.5814497,0.9309884,-0.126061083,blue,0.03193255
ADAM22,0.49732207,0.40766966,0.71475197,0.9948641,0.4076697,0.9772987,-0.052744263,blue,0.03193255
SERPINB2,0.21334828,0.24729718,0.33722255,0.9948641,0.2472972,0.8724373,-0.008123245,blue,0.03193255
A2ML1,0.06098362,0.03802438,0.03802438,0.9948641,0.2032537,0.9591136,-0.054537891,blue,0.03193255
LGALS9C,0.73031118,0.50005620,0.50005620,0.9948641,0.6710236,0.9168106,0.022294152,blue,0.03193255
GLDN,0.52123208,0.38916783,0.38916783,0.9948641,0.4447603,0.9150983,0.033854966,blue,0.03193255
LGALS7,0.22872063,0.13176989,0.13176989,0.9948641,0.2430773,0.9323875,-0.064439173,blue,0.03193255
SERPINB12,0.31224182,0.11228683,0.11228683,0.9948641,0.3857235,0.9591136,-0.030693233,blue,0.03193255
BMP3,0.04411467,0.06938372,0.06938372,0.9948641,0.2983555,0.7679758,0.019218565,blue,0.03193255


In [7]:
if (save_res) {
    meta_df %>% write_tsv(paste0(dirs$analysis_dir, "/one_off/cesc_figo_full.tsv"))
}